In [1]:
import pickle

import numpy as np
import pandas as pd

from cyclops.processors.aggregate import Aggregator
from cyclops.processors.cleaning import (
    normalize_categories,
    normalize_names,
    normalize_values,
)
from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    AGE,
    CARE_UNIT,
    DIAGNOSIS_CODE,
    DIAGNOSIS_TRAJECTORY,
    ENCOUNTER_ID,
    EVENT_CATEGORY,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_VALUE_UNIT,
    HOSPITAL_ID,
    SEX,
    SUBJECT_ID,
    TIMESTEP,
    YEAR,
)
from cyclops.processors.constants import (
    BINARY,
    BY,
    CATEGORICAL_INDICATOR,
    FEATURE_INDICATOR_ATTR,
    FEATURE_MAPPING_ATTR,
    FEATURE_TYPE_ATTR,
    FEATURE_TYPES,
    FEATURES,
    MEAN,
    MIN_MAX,
    MISSING_CATEGORY,
    NUMERIC,
    ORDINAL,
    STANDARD,
    STRING,
    TARGETS,
)
from cyclops.processors.feature.feature import TabularFeatures, TemporalFeatures
from cyclops.processors.feature.normalization import VectorizedNormalizer
from cyclops.processors.feature.split import split_idx
from cyclops.query import mimic
from cyclops.query import process as qp
from cyclops.utils.common import print_dict
from cyclops.utils.file import load_array, load_dataframe, save_dataframe

2022-07-15 12:17:25,915 INFO cyclops.orm     - Database setup, ready to run queries!


In [2]:
YEARS = [2015, 2016, 2018, 2019, 2020]
MIN_YEAR = min(YEARS)
TARGET_NAME = "death"

# Querying

## Patient encounters

In [3]:
encounters_interface = mimic.patient_encounters(
    years=YEARS, died=True, died_binarize_col="died"
)
encounters_query = encounters_interface.query
encounters_query = qp.Drop(
    ["insurance", "language", "marital_status", "hospital_expire_flag"]
)(encounters_query)
encounters_interface = mimic.get_interface(encounters_query)
encounters = encounters_interface.run()
encounters

2022-07-15 12:17:28,317 INFO cyclops.orm     - Query returned successfully!
2022-07-15 12:17:28,318 INFO cyclops.utils.profile - Finished executing function run_query in 2.349815 s


,subject_id,encounter_id,admit_timestamp,discharge_timestamp,deathtime,admission_type,admission_location,discharge_location,ethnicity,edregtime,edouttime,sex,age,birth_year,dod,anchor_year_difference,died
0,16115393,21765983,2015-02-22 18:52:00,2015-03-05 10:10:00,NaT,ELECTIVE,None,HOME,UNKNOWN,NaT,NaT,M,0,2015,NaT,-157,False
1,18373085,24514056,2015-10-20 11:22:00,2015-10-23 12:40:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,M,0,2015,NaT,-125,False
2,16591664,22764393,2015-03-02 04:41:00,2015-03-03 13:30:00,NaT,ELECTIVE,None,HOME,HISPANIC/LATINO,NaT,NaT,F,0,2015,NaT,-150,False
3,14083263,26476486,2015-01-28 02:01:00,2015-01-29 14:45:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,M,0,2015,NaT,-101,False
4,12067589,23047540,2018-05-20 02:21:00,2018-05-22 11:57:00,NaT,ELECTIVE,None,HOME,ASIAN,NaT,NaT,M,0,2018,NaT,-124,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224759,11601417,29160491,2015-05-23 00:00:00,2015-05-27 13:55:00,NaT,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,WHITE,NaT,NaT,F,55,1960,NaT,-117,False
224760,19139690,24101028,2018-06-24 02:05:00,2018-06-24 15:38:00,NaT,EU OBSERVATION,WALK-IN/SELF REFERRAL,HOME,WHITE,2174-06-23 23:56:00,2174-06-24 03:10:00,F,27,1991,NaT,-156,False
224761,10530550,27104238,2018-12-04 00:00:00,2018-12-08 10:40:00,NaT,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,WHITE,NaT,NaT,M,71,1947,NaT,-150,False
224762,10934846,20547990,2016-10-26 08:00:00,2016-10-28 17:47:00,NaT,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,WHITE,NaT,NaT,F,58,1958,NaT,-171,False


In [4]:
save_dataframe(encounters, "encounters.parquet")

2022-07-15 12:17:28,342 INFO cyclops.utils.file - Saving dataframe to encounters.parquet


'encounters.parquet'

## Events

In [5]:
events_interface = mimic.events()
events_query = events_interface.query
events_query = qp.Drop(["warning", "itemid", "storetime"])(events_query)
events_interface = mimic.get_interface(events_query)
events = events_interface.run(limit=1000000)

# Reverse the deidentified dating
events = pd.merge(
    encounters[[ENCOUNTER_ID, "anchor_year_difference"]], events, on=ENCOUNTER_ID
)


def add_offset(row):
    row[EVENT_TIMESTAMP] += pd.DateOffset(years=row["anchor_year_difference"])
    return row


events = events.apply(add_offset, axis=1)
events = events.drop("anchor_year_difference", axis=1)

2022-07-15 12:17:33,181 INFO cyclops.orm     - Query returned successfully!
2022-07-15 12:17:33,182 INFO cyclops.utils.profile - Finished executing function run_query in 4.530989 s


In [6]:
# Create the target as a timeseries event
target_events = encounters[encounters["died"] == True]
target_events = target_events[[ENCOUNTER_ID, "deathtime"]]
target_events = target_events.rename({"deathtime": EVENT_TIMESTAMP}, axis=1)
target_events[EVENT_NAME] = TARGET_NAME
target_events[EVENT_CATEGORY] = TARGETS
target_events[EVENT_VALUE] = 1
target_events.head(5)

,encounter_id,event_timestamp,event_name,event_category,event_value
1572,24947486,2015-06-14 17:45:00,death,targets,1
1595,25426484,2018-02-21 08:00:00,death,targets,1
1655,22176511,2018-11-30 22:50:00,death,targets,1
1672,29673314,2015-12-14 08:35:00,death,targets,1
1705,21444472,2015-03-02 00:00:00,death,targets,1


In [7]:
# Include target
events = pd.concat([events, target_events])

In [8]:
# Preprocessing
events[EVENT_NAME] = normalize_names(events[EVENT_NAME])
events[EVENT_CATEGORY] = normalize_categories(events[EVENT_CATEGORY])
# events[EVENT_VALUE] = normalize_values(events[EVENT_VALUE])

# Concatenate event name and category since some names are the same in
# different categories, e.g., 'flow' for categories 'heartware' and 'ecmo'
events[EVENT_NAME] = events[EVENT_CATEGORY] + " - " + events[EVENT_NAME]
events.head(5)

# Update target name having included the category
TARGET_NAME = TARGETS + " - " + TARGET_NAME

In [9]:
save_dataframe(events, "events.parquet")

2022-07-15 12:18:30,151 INFO cyclops.utils.file - Saving dataframe to events.parquet


'events.parquet'

------------------------------------------------------------------------------------------------

# Processing

## Tabular

In [10]:
encounters = load_dataframe("encounters.parquet")
encounters.head(5)

2022-07-15 12:18:30,450 INFO cyclops.utils.file - Loading DataFrame from encounters.parquet


,subject_id,encounter_id,admit_timestamp,discharge_timestamp,deathtime,admission_type,admission_location,discharge_location,ethnicity,edregtime,edouttime,sex,age,birth_year,dod,anchor_year_difference,died
0,16115393,21765983,2015-02-22 18:52:00,2015-03-05 10:10:00,NaT,ELECTIVE,None,HOME,UNKNOWN,NaT,NaT,M,0,2015,NaT,-157,False
1,18373085,24514056,2015-10-20 11:22:00,2015-10-23 12:40:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,M,0,2015,NaT,-125,False
2,16591664,22764393,2015-03-02 04:41:00,2015-03-03 13:30:00,NaT,ELECTIVE,None,HOME,HISPANIC/LATINO,NaT,NaT,F,0,2015,NaT,-150,False
3,14083263,26476486,2015-01-28 02:01:00,2015-01-29 14:45:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,M,0,2015,NaT,-101,False
4,12067589,23047540,2018-05-20 02:21:00,2018-05-22 11:57:00,NaT,ELECTIVE,None,HOME,ASIAN,NaT,NaT,M,0,2018,NaT,-124,False


In [11]:
features = [
    AGE,
    SEX,
    "admission_type",
    "admission_location",
    "discharge_location",
    "ethnicity",
]

tab_features = TabularFeatures(
    data=encounters,
    features=features,
    by=ENCOUNTER_ID,
)

In [12]:
tab_features.types

{'admission_location': 'ordinal',
 'age': 'numeric',
 'discharge_location': 'ordinal',
 'sex': 'binary',
 'admission_type': 'ordinal',
 'ethnicity': 'ordinal'}

In [13]:
tab_features.meta["admission_location"].get_mapping()

{0: 'AMBULATORY SURGERY TRANSFER',
 1: 'CLINIC REFERRAL',
 2: 'EMERGENCY ROOM',
 3: 'INFORMATION NOT AVAILABLE',
 4: 'INTERNAL TRANSFER TO OR FROM PSYCH',
 5: 'PACU',
 6: 'PHYSICIAN REFERRAL',
 7: 'PROCEDURE SITE',
 8: 'TRANSFER FROM HOSPITAL',
 9: 'TRANSFER FROM SKILLED NURSING FACILITY',
 10: 'WALK-IN/SELF REFERRAL',
 11: 'nan'}

In [14]:
ordinal_features = tab_features.features_by_type(ORDINAL)
ordinal_features

['admission_location', 'discharge_location', 'admission_type', 'ethnicity']

In [15]:
tab_vectorized = tab_features.vectorize(to_binary_indicators=ordinal_features)

/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat] = data[feat].replace(self.meta[feat].get_mapping())
/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat] = data[feat].replace(self.meta[feat].get_mapping())
/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [16]:
tab_vectorized.shape

(224764, 43)

In [17]:
tab_vectorized.axis_names

['encounter_id', 'features']

In [18]:
tab_vectorized.get_index(FEATURES)

array(['sex', 'age', 'admission_location_AMBULATORY SURGERY TRANSFER',
       'admission_location_CLINIC REFERRAL',
       'admission_location_EMERGENCY ROOM',
       'admission_location_INFORMATION NOT AVAILABLE',
       'admission_location_INTERNAL TRANSFER TO OR FROM PSYCH',
       'admission_location_PACU', 'admission_location_PHYSICIAN REFERRAL',
       'admission_location_PROCEDURE SITE',
       'admission_location_TRANSFER FROM HOSPITAL',
       'admission_location_TRANSFER FROM SKILLED NURSING FACILITY',
       'admission_location_WALK-IN/SELF REFERRAL',
       'discharge_location_ACUTE HOSPITAL',
       'discharge_location_AGAINST ADVICE',
       'discharge_location_ASSISTED LIVING',
       'discharge_location_CHRONIC/LONG TERM ACUTE CARE',
       'discharge_location_DIED',
       'discharge_location_HEALTHCARE FACILITY',
       'discharge_location_HOME', 'discharge_location_HOME HEALTH CARE',
       'discharge_location_HOSPICE', 'discharge_location_OTHER FACILITY',
       'di

In [19]:
numeric_features = tab_features.features_by_type(NUMERIC)
numeric_features

['age']

## Temporal

In [20]:
events = load_dataframe("events.parquet")

events = events.sample(n=int(len(events) / 20))

events.head(5)

2022-07-15 12:18:31,917 INFO cyclops.utils.file - Loading DataFrame from events.parquet


,encounter_id,subject_id,stay_id,event_timestamp,value,event_value,event_value_unit,event_category,event_name
409643,26145732,16454295.0,33444947.0,2015-04-07 08:00:00,Strong,NaN,None,pulmonary,pulmonary - cough effort
455523,21282089,16486338.0,38142994.0,2018-10-30 02:00:00,Adequate lighting,NaN,None,restraint/support systems,restraint/support systems - less restrictive m...
279048,24502012,16481227.0,31419485.0,2018-01-05 02:00:00,No,NaN,None,pain/sedation,pain/sedation - pain present
531267,22514759,16528460.0,33558202.0,2019-08-12 10:00:00,Calm,NaN,None,restraint/support systems,restraint/support systems - status and comfort
401443,23267387,16528873.0,33747740.0,2020-12-22 20:00:00,Heel/Elbow Pads,NaN,None,treatments,treatments - pressure reducing device


In [21]:
# Keep only the most popular events
top_events = events[EVENT_NAME].value_counts()[:100].index

# Force include the target
top_events = np.unique(np.append(top_events, TARGET_NAME))

events = events[events[EVENT_NAME].isin(top_events)]
events.head(5)

,encounter_id,subject_id,stay_id,event_timestamp,value,event_value,event_value_unit,event_category,event_name
409643,26145732,16454295.0,33444947.0,2015-04-07 08:00:00,Strong,NaN,None,pulmonary,pulmonary - cough effort
455523,21282089,16486338.0,38142994.0,2018-10-30 02:00:00,Adequate lighting,NaN,None,restraint/support systems,restraint/support systems - less restrictive m...
279048,24502012,16481227.0,31419485.0,2018-01-05 02:00:00,No,NaN,None,pain/sedation,pain/sedation - pain present
531267,22514759,16528460.0,33558202.0,2019-08-12 10:00:00,Calm,NaN,None,restraint/support systems,restraint/support systems - status and comfort
401443,23267387,16528873.0,33747740.0,2020-12-22 20:00:00,Heel/Elbow Pads,NaN,None,treatments,treatments - pressure reducing device


In [22]:
events[events[EVENT_NAME] == TARGET_NAME].head(5)

,encounter_id,subject_id,stay_id,event_timestamp,value,event_value,event_value_unit,event_category,event_name
19294,24531789,NaN,NaN,2018-12-17 21:54:00,None,1.0,None,targets,targets - death
93956,24340569,NaN,NaN,2015-11-30 19:50:00,None,1.0,None,targets,targets - death
221526,23596983,NaN,NaN,2015-06-29 10:50:00,None,1.0,None,targets,targets - death
147295,22270906,NaN,NaN,2015-04-29 16:25:00,None,1.0,None,targets,targets - death
117537,28992436,NaN,NaN,2015-10-31 09:42:00,None,1.0,None,targets,targets - death


In [23]:
aggregator = Aggregator(
    aggfuncs={EVENT_VALUE: MEAN},
    timestamp_col=EVENT_TIMESTAMP,
    time_by=ENCOUNTER_ID,
    agg_by=[ENCOUNTER_ID, EVENT_NAME],
    timestep_size=8,
    window_duration=24,  # Optional
    # agg_meta_for=EVENT_VALUE,  # Optional
)

In [24]:
events = events.reset_index(drop=True)

tmp_features = TemporalFeatures(
    events,
    features=EVENT_VALUE,
    by=[ENCOUNTER_ID, EVENT_NAME],
    timestamp_col=EVENT_TIMESTAMP,
    aggregator=aggregator,
)

In [25]:
aggregated = tmp_features.aggregate()
aggregated.head(5)

2022-07-15 12:18:32,208 INFO cyclops.processors.cleaning - Dropped nulls over columns: event_timestamp. Removed 3 rows.
2022-07-15 12:18:35,858 INFO cyclops.utils.profile - Finished executing function __call__ in 3.653941 s


event_value  \
encounter_id event_name                             timestep                
20020590     targets - death                        0                 1.0   
20068018     routine vital signs - temperature site 0                 NaN   
             restraint/support systems - side rails 0                 NaN   
             routine vital signs - ectopy type 1    2                 NaN   
             treatments - activity tolerance        1                 NaN   

                                                                  timestep_start  
encounter_id event_name                             timestep                      
20020590     targets - death                        0        2015-11-19 18:00:00  
20068018     routine vital signs - temperature site 0        2015-07-21 20:53:00  
             restraint/support systems - side rails 0        2015-07-21 20:53:00  
             routine vital signs - ectopy type 1    2        2015-07-22 12:53:00  
             treatments - activity tolerance        1        2015-07-22 04:53:00

In [26]:
save_dataframe(aggregated, "aggregated.parquet")

2022-07-15 12:18:35,868 INFO cyclops.utils.file - Saving dataframe to aggregated.parquet


'aggregated.parquet'

In [27]:
temp_vectorized = aggregator.vectorize(aggregated)
temp_vectorized.shape

2022-07-15 12:18:36,580 INFO cyclops.utils.profile - Finished executing function vectorize in 0.696335 s


(1, 346, 100, 3)

In [28]:
temp_vectorized.axis_names

['aggfuncs', 'encounter_id', 'event_name', 'timesteps']

## Dataset splits

Take only the encounters available in all of the datasets and align the datasets over encounters.

In [29]:
tab_encounters = set(tab_vectorized.get_index(ENCOUNTER_ID))
temp_encounters = set(temp_vectorized.get_index(ENCOUNTER_ID))
encounters_intersection = np.array(list(tab_encounters.intersection(temp_encounters)))

In [30]:
tab_vectorized = tab_vectorized.take_with_index(ENCOUNTER_ID, encounters_intersection)
temp_vectorized = temp_vectorized.take_with_index(ENCOUNTER_ID, encounters_intersection)

In [31]:
tab_vectorized.shape

(346, 43)

In [32]:
temp_vectorized.shape

(1, 346, 100, 3)

Split into training, validation, and testing datasets.

Split using the same indices such that the tabular and temporal datasets remain aligned.

In [33]:
splits = split_idx(
    fractions=[0.8, 0.1, 0.1],
    data_len=len(encounters_intersection),
)

In [34]:
tab_train_X, tab_val_X, tab_test_X = tab_vectorized.split_by_indices(
    ENCOUNTER_ID, splits
)

temp_train, temp_val, temp_test = temp_vectorized.split_by_indices(ENCOUNTER_ID, splits)

In [35]:
tab_train_X.shape, tab_val_X.shape, tab_test_X.shape

((277, 43), (34, 43), (35, 43))

In [36]:
temp_train.shape, temp_val.shape, temp_test.shape

((1, 277, 100, 3), (1, 34, 100, 3), (1, 35, 100, 3))

### Split features/targets

Split out the targets in the temporal data.

In [37]:
temp_train_X, temp_train_y = temp_train.split_out(EVENT_NAME, [TARGET_NAME])
temp_train_X.shape, temp_train_y.shape

((1, 277, 99, 3), (1, 277, 1, 3))

In [38]:
temp_val_X, temp_val_y = temp_val.split_out(EVENT_NAME, [TARGET_NAME])
temp_val_X.shape, temp_val_y.shape

((1, 34, 99, 3), (1, 34, 1, 3))

In [39]:
temp_test_X, temp_test_y = temp_test.split_out(EVENT_NAME, [TARGET_NAME])
temp_test_X.shape, temp_test_y.shape

((1, 35, 99, 3), (1, 35, 1, 3))

### Normalization

In [40]:
splits = tab_train_X, tab_val_X, tab_test_X

for split in splits:
    normalizer = VectorizedNormalizer(
        split.get_axis(FEATURES),
        normalizer_map={feat: STANDARD for feat in numeric_features},
    )
    split.add_normalizer(normalizer)
    split.normalize()

tab_train_X, tab_val_X, tab_test_X = splits

In [41]:
# Only normalizing the inputs since the outputs are binary
splits = temp_train_X, temp_val_X, temp_test_X

for split in splits:
    normalizer = VectorizedNormalizer(
        split.get_axis(EVENT_NAME),
        normalization_method=STANDARD,
    )
    split.add_normalizer(normalizer)
    split.normalize()

temp_train_X, temp_val_X, temp_test_X = splits

In [42]:
"Percentage null:", np.isnan(temp_train_X.data).sum() / temp_train_X.data.size

('Percentage null:', 0.9946638466494062)

## Save

In [43]:
tab_train_X.save("tab_train_X.npy")
tab_val_X.save("tab_val_X.npy")
tab_test_X.save("tab_test_X.npy")

temp_train_X.save("temp_train_X.npy")
temp_val_X.save("temp_val_X.npy")
temp_test_X.save("temp_test_X.npy")

temp_train_y.save("temp_train_y.npy")
temp_val_y.save("temp_val_y.npy")
temp_test_y.save("temp_test_y.npy")

2022-07-15 12:18:36,880 INFO cyclops.utils.file - Saving array to tab_train_X.npy
2022-07-15 12:18:36,882 INFO cyclops.utils.file - Saving array to tab_val_X.npy
2022-07-15 12:18:36,884 INFO cyclops.utils.file - Saving array to tab_test_X.npy
2022-07-15 12:18:36,885 INFO cyclops.utils.file - Saving array to temp_train_X.npy
2022-07-15 12:18:36,887 INFO cyclops.utils.file - Saving array to temp_val_X.npy
2022-07-15 12:18:36,888 INFO cyclops.utils.file - Saving array to temp_test_X.npy
2022-07-15 12:18:36,890 INFO cyclops.utils.file - Saving array to temp_train_y.npy
2022-07-15 12:18:36,891 INFO cyclops.utils.file - Saving array to temp_val_y.npy
2022-07-15 12:18:36,892 INFO cyclops.utils.file - Saving array to temp_test_y.npy


'temp_test_y.npy'